# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,497.105798,-0.695356,1234,496.410443,497.801154,992.820885,995.602309
6,527.301523,0.217814,1234,527.519337,527.083709,1055.038674,1054.167419
7,871.524276,0.444577,1234,871.968854,871.079699,1743.937707,1742.159398
8,544.321323,0.380770,1234,544.702094,543.940553,1089.404187,1087.881106
9,470.328747,0.503414,1234,470.832161,469.825333,941.664322,939.650666
10,409.856551,0.155864,1234,410.012416,409.700687,820.024831,819.401375
11,976.378955,-0.985633,1234,975.393322,977.364588,1950.786644,1954.729177
12,259.707573,0.508828,1234,260.216401,259.198745,520.432803,518.397490
13,458.528208,-0.966993,1234,457.561215,459.495202,915.122430,918.990403
14,547.373894,-0.252121,1234,547.121773,547.626016,1094.243545,1095.252031


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-496.410443,497.801154
6,-527.519337,527.083709
7,-871.968854,871.079699
8,-544.702094,543.940553
9,-470.832161,469.825333
10,-410.012416,409.700687
11,-975.393322,977.364588
12,-260.216401,259.198745
13,-457.561215,459.495202
14,-547.121773,547.626016


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-496.410443,497.801154
6,-527.519337,527.083709
7,-871.968854,871.079699
8,-544.702094,543.940553
9,-470.832161,469.825333
10,-410.012416,409.700687
11,-975.393322,977.364588
12,-260.216401,259.198745
13,-457.561215,459.495202
14,-547.121773,547.626016


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-496.4104425 ,  497.80115447],
       [-527.51933716,  527.08370939],
       [-871.96885363,  871.07969898],
       [-544.70209367,  543.94055314],
       [-470.83216122,  469.82533289],
       [-410.01241556,  409.70068737],
       [-975.39332196,  977.36458831],
       [-260.21640139,  259.1987448 ],
       [-457.56121504,  459.4952016 ],
       [-547.12177269,  547.62601564]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,990.820885,-995.602309
6,1053.038674,-1054.167419
7,1741.937707,-1742.159398
8,1087.404187,-1087.881106
9,939.664322,-939.650666
10,818.024831,-819.401375
11,1948.786644,-1954.729177
12,518.432803,-518.397490
13,913.122430,-918.990403
14,1092.243545,-1095.252031


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.497106,0.152322
6,0.527302,0.608907
7,0.871524,0.722289
8,0.544321,0.690385
9,0.470329,0.751707
10,0.409857,0.577932
11,0.976379,0.007183
12,0.259708,0.754414
13,0.458528,0.016503
14,0.547374,0.373939


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,497.105798,-0.695356
6,527.301523,0.217814
7,871.524276,0.444577
8,544.321323,0.380770
9,470.328747,0.503414
10,409.856551,0.155864
11,976.378955,-0.985633
12,259.707573,0.508828
13,458.528208,-0.966993
14,547.373894,-0.252121


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
